# Using cloudknot to perform matrix-vector multiplication of random matrices

This example uses cloudknot to perform matrix-vector multiplication of some random matrices with varying standard deviations.

In [1]:
import cloudknot as ck

First, we write the python script that we want to run on AWS batch. Note that we import the necessary python packages within the function `random_mv_prod`.

In [2]:
def random_mv_prod(b):
    import numpy as np
    
    x = np.random.normal(0, b, 1024)
    A = np.random.normal(0, b, (1024, 1024))
    
    return np.dot(A, x)

Create a knot using the `random_mv_prod` function and a job definition memory of 128 MiB.

In [3]:
try:
    knot = ck.Knot(name='random_mv_prod', func=random_mv_prod, memory=128, retries=3)
except ValueError:
    # If you previously created this knot but didn't clobber it,
    # then this except clause will try to retrieve it
    knot = ck.Knot(name='random_mv_prod')

Submit 20 batch jobs to the knot. The `map()` method returns a list of futures for the results of each batch job. You can optionally supply a list of environment variables to each job.

In [5]:
# import numpy since it was only imported in the `random_mv_prod` function above
import numpy as np

In [4]:
# Submit the jobs
result_futures = knot.map(np.linspace(0.1, 100, 20), env_vars=[{'name': 'MY_ENV_VAR', 'value': 'foo'}])

We can query the jobs associated with this knot by calling `knot.view_jobs()`, prints a bunch of job info and provides a consice summary of job statuses.

In [15]:
# Rerun this cell as often as you like to update your job status info
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
57e9d725-3aab-4e1c-859a-53b06ff18ec8        random_mv_prod-12           SUCCEEDED
c9b01618-a20a-4794-96aa-7b6900d39a43        random_mv_prod-18           SUCCEEDED
ae852c96-cc1b-48fe-911e-f1d7197ff341        random_mv_prod-0            SUCCEEDED
ca0f4589-e5bc-42b6-af5a-119c46644c24        random_mv_prod-15           SUCCEEDED
a8a941fe-12bf-4ac7-a4f7-70e84c577321        random_mv_prod-3            SUCCEEDED
0e7af839-3a91-49c2-b8d4-3d990f697b36        random_mv_prod-6            SUCCEEDED
acbdf052-dcc8-4df1-9095-2ed4f52e66dc        random_mv_prod-10           SUCCEEDED
2c063597-36de-4273-b074-6e89d61e48e5        random_mv_prod-16           SUCCEEDED
95857b21-ed97-4fcc-9fbc-66069906a9c3        random_mv_prod-2            SUCCEEDED
cdd72224-6368-4a71-8bdd-d732212c67b7        random_mv_prod-7            SUCCEEDED
d07f1371-53c7-47d1-a84e-2ad9caffcf10        random_mv_prod-17   

We can also inspect each BatchJob instance by looking at `knot.jobs` which returns a list of BatchJob instances for each submitted job, e.g.:

In [16]:
last_job = knot.jobs[-1]

In [8]:
print(last_job.done)
print(last_job.result(timeout=5))

False


CKTimeoutError: The job with job-id b873b0b5-4bdc-46b4-91f5-b6e6266f56b2 did not finish within the requested timeout period

`Knot.map()` returns a list of futures so you can use any of the futures methods to query the results, e.g. `done()` or `result()`.

In [17]:
print(result_futures[0].done())
print(result_futures[0].result())

True
[ 0.1824538   0.13255699 -0.32016405 ..., -0.48144142  0.18720769
  0.09038733]


Once you're all done, clobber the knot, including the underlying PARS and the remote repo.

In [19]:
knot.clobber(clobber_pars=True, clobber_repo=True, clobber_image=True)